Attempt to solve the problem in 2 stages. First, try to determine if this is a whale we've seen before. Secondly for each whale we have seen before, match the tail to an id.

# Pre-trained model

In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Concatenate
import os

# Add new layers

In [2]:
import numpy as np
import pandas as pd

train_map_df = pd.read_csv(os.path.join("data", "train.csv"))
train_map_df['Id'] = train_map_df['Id'].astype('category')
train_map_df['is_new'] = np.where(train_map_df['Id'] == 'new_whale', 'new', 'existing')
train_map_df.head()

,Image,Id,is_new
0,0000e88ab.jpg,w_f48451c,existing
1,0001f9222.jpg,w_c3d896a,existing
2,00029d126.jpg,w_20df2c5,existing
3,00050a15a.jpg,new_whale,new
4,0005c1ef8.jpg,new_whale,new


In [3]:
inputs = keras.Input(shape=(224, 224, 3))


base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)
base_model.summary()
base_model.trainable = False


x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x) # A Dense classifier with a class for new/existing

model = keras.Model(inputs, outputs)


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


# Compiling Model

In [5]:
# we have a binary problem (either new or not new)
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

# Augment the Data

In [6]:
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,
        vertical_flip=False,
        validation_split=0.2)  # 80/20 split train/test

In [7]:
train_it = datagen.flow_from_dataframe(train_map_df,
                                       directory=os.path.join('data', 'train'), 
                                       x_col='Image',
                                       y_col='is_new',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8,
                                       subset='training')
valid_it = datagen.flow_from_dataframe(train_map_df,
                                       directory=os.path.join('data', 'train'), 
                                       x_col='Image',
                                       y_col='is_new',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8,
                                       subset='validation')

Found 20289 validated image filenames belonging to 2 classes.
Found 5072 validated image filenames belonging to 2 classes.


In [8]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=50)

Epoch 1/50
12/12 [==============================] - 32s 3s/step - loss: 1.3898 - binary_accuracy: 0.5521 - val_loss: 1.0892 - val_binary_accuracy: 0.4375
Epoch 2/50
12/12 [==============================] - 29s 2s/step - loss: 1.3359 - binary_accuracy: 0.5729 - val_loss: 1.3357 - val_binary_accuracy: 0.5625
Epoch 3/50
12/12 [==============================] - 27s 2s/step - loss: 1.3092 - binary_accuracy: 0.5000 - val_loss: 0.8758 - val_binary_accuracy: 0.7188
Epoch 4/50
12/12 [==============================] - 24s 2s/step - loss: 1.5318 - binary_accuracy: 0.5104 - val_loss: 1.1720 - val_binary_accuracy: 0.5312
Epoch 5/50
12/12 [==============================] - 23s 2s/step - loss: 0.8838 - binary_accuracy: 0.6667 - val_loss: 0.9018 - val_binary_accuracy: 0.5625
Epoch 6/50
12/12 [==============================] - 22s 2s/step - loss: 1.2132 - binary_accuracy: 0.5729 - val_loss: 1.4405 - val_binary_accuracy: 0.6250
Epoch 7/50
12/12 [==============================] - 21s 2s/step - loss: 1.21

In [9]:
train_map_df['full_path'] = train_map_df['Image'].apply(lambda x: os.path.join('data', 'train', x))
train_map_df.head()

,Image,Id,is_new,full_path
0,0000e88ab.jpg,w_f48451c,existing,data\train\0000e88ab.jpg
1,0001f9222.jpg,w_c3d896a,existing,data\train\0001f9222.jpg
2,00029d126.jpg,w_20df2c5,existing,data\train\00029d126.jpg
3,00050a15a.jpg,new_whale,new,data\train\00050a15a.jpg
4,0005c1ef8.jpg,new_whale,new,data\train\0005c1ef8.jpg


In [16]:
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

predictions = []
paths = train_map_df['Image']

def make_prediction(image_path):
    image = image_utils.load_img(os.path.join('data', 'train', image_path), target_size=(224, 224), color_mode="rgb")
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224, 3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds[0][0]

for id in train_map_df['Image']:
    predictions.append(0 if make_prediction(id) < 0 else 1)

train_map_df['predicted_new'] = pd.Series(predictions)
train_map_df

,Image,Id,is_new,full_path,predicted_new
0,0000e88ab.jpg,w_f48451c,existing,data\train\0000e88ab.jpg,0
1,0001f9222.jpg,w_c3d896a,existing,data\train\0001f9222.jpg,1
2,00029d126.jpg,w_20df2c5,existing,data\train\00029d126.jpg,0
3,00050a15a.jpg,new_whale,new,data\train\00050a15a.jpg,0
4,0005c1ef8.jpg,new_whale,new,data\train\0005c1ef8.jpg,0
...,...,...,...,...,...
25356,ffef89eed.jpg,w_9c506f6,existing,data\train\ffef89eed.jpg,0
25357,fff7faf61.jpg,w_9cf0388,existing,data\train\fff7faf61.jpg,0
25358,fff9002e0.jpg,w_bd1c3d5,existing,data\train\fff9002e0.jpg,0
25359,fffcde6fe.jpg,w_9f30885,existing,data\train\fffcde6fe.jpg,0


Secondary model to id existing whales.

In [17]:
inputs_2 = keras.Input(shape=(224, 224, 3))
x_2 = base_model(inputs_2, training=False)
x_2 = keras.layers.GlobalAveragePooling2D()(x_2)
outputs_2 = keras.layers.Dense(len(train_map_df['Id'].unique())-1)(x_2) # A Dense classifier with a each class from our training set excluded new_whale


base_model_2 = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

base_model_2.summary()
base_model_2.trainable = False

model_2 = keras.Model(inputs_2, outputs_2)

# we have a categorical problem so need to use CategoricalCrossentropy and CategoricalAccuracy
model_2.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=[keras.metrics.CategoricalAccuracy()])

datagen_2 = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,
        vertical_flip=False,
        validation_split=0.2)  # 80/20 split train/test

train_it_2 = datagen.flow_from_dataframe(train_map_df[train_map_df['is_new'] == 'existing'],
                                       directory=os.path.join('data', 'train'), 
                                       x_col='Image',
                                       y_col='Id',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='categorical', 
                                       batch_size=8,
                                       subset='training')
valid_it_2 = datagen.flow_from_dataframe(train_map_df[train_map_df['is_new'] == 'existing'],
                                       directory=os.path.join('data', 'train'), 
                                       x_col='Image',
                                       y_col='Id',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='categorical', 
                                       batch_size=8,
                                       subset='validation')
model_2.fit(train_it_2, steps_per_epoch=12, validation_data=valid_it_2, validation_steps=4, epochs=50)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

Calculate accuracy

In [18]:
predictions = []
existing_ids = train_map_df['Id'].unique()

for image, predicted_new in zip(train_map_df['Image'], train_map_df['predicted_new']):
    if predicted_new == 1:
        predictions.append('new_whale')
    else:
        image = image_utils.load_img(os.path.join('data', 'train', image), target_size=(224, 224), color_mode="rgb")
        image = image_utils.img_to_array(image)
        image = image.reshape(1,224,224, 3)
        image = preprocess_input(image)
        preds = model_2.predict(image)
        predictions.append(existing_ids[np.argmax(preds)])
        
train_map_df['predictions'] = pd.Series(predictions)
train_map_df

,Image,Id,is_new,full_path,predicted_new,predictions
0,0000e88ab.jpg,w_f48451c,existing,data\train\0000e88ab.jpg,0,w_75cebcf
1,0001f9222.jpg,w_c3d896a,existing,data\train\0001f9222.jpg,1,new_whale
2,00029d126.jpg,w_20df2c5,existing,data\train\00029d126.jpg,0,w_75cebcf
3,00050a15a.jpg,new_whale,new,data\train\00050a15a.jpg,0,w_59d1835
4,0005c1ef8.jpg,new_whale,new,data\train\0005c1ef8.jpg,0,w_56cb095
...,...,...,...,...,...,...
25356,ffef89eed.jpg,w_9c506f6,existing,data\train\ffef89eed.jpg,0,w_75cebcf
25357,fff7faf61.jpg,w_9cf0388,existing,data\train\fff7faf61.jpg,0,w_75cebcf
25358,fff9002e0.jpg,w_bd1c3d5,existing,data\train\fff9002e0.jpg,0,w_75cebcf
25359,fffcde6fe.jpg,w_9f30885,existing,data\train\fffcde6fe.jpg,0,w_30a7b3f


In [19]:
print('Total accuracy: ' + str(np.sum(train_map_df['Id'] == train_map_df['predictions']) / len(train_map_df['Id'])))

Total accuracy: 0.11584716690982216


In [20]:
print('Naive accuracy: ' + str(np.sum(train_map_df['Id'] == 'new_whale') / len(train_map_df['Id'])))

Naive accuracy: 0.3810575292772367
